## Web信息收集
### HTML
`read_html`：对html文件中的表格进行解析，得到的由DataFrame组成的列表，列表中每个DataFrame对应html文件中的每个表格
> 注：
- html文件必须是标准的html5格式，表格列名要用<thead\>标签包围，否则生成的DataFrame是不会将表格列名作为columns的，而是用自动生成的数字序列代替；
- 若html文件表格中有<tbody\>标签，则数据内容需要包含在<tbody\>标签中，否则不会被读取；
- 若没有<tbody\>标签，表格所有数据都会被读取为DataFrame的values，如果此时又有<thead\>标签，则列名会被同时读取为columns和values，即columns和第一行内容都是列名。

参数：  
`header`：指定行索引所对应的行作为columns，对于没有<thead\>标签的表格，可使用`header=0`  
`index_col`：指定列索引所对应的列作为index

In [1]:
import pandas as pd

In [2]:
tables=pd.read_html('doc\pd-web.html')

In [3]:
table=tables[0]

In [4]:
print("tables' type is %s\ntable's type is %s"%(type(tables),type(table)))

tables' type is <class 'list'>
table's type is <class 'pandas.core.frame.DataFrame'>


In [5]:
table.head()

,箱号,尺寸,箱型,进场日期,堆存天数,箱动态,箱状况,来箱情况,备注,堆场
0,AMFU3238167,20,GP,2019-1-20 13:05:44,27,EC,坏箱,回空,NaN,SHIC
1,BMOU2241483,20,GP,2019-1-28 20:45:05,19,EC,坏箱,回空,NaN,SHIC
2,BMOU2405247,20,GP,2019-1-22 21:04:15,25,EC,好箱,洋山三期,NaN,SHWL
3,BMOU2552042,20,GP,2019-1-27 10:24:36,20,EC,坏箱,回空,NaN,SHIC
4,BMOU2553711,20,GP,2019-2-15 13:47:58,1,EC,坏箱,回空,NaN,SHIC


In [6]:
table['箱号'][1:4] # 指定某列某行，得到个Series

1    BMOU2241483
2    BMOU2405247
3    BMOU2552042
Name: 箱号, dtype: object

In [7]:
# pd-web-1.html的表格有<thead>但没有<tbody>，列名会同时被读取为columns和values
t1=pd.read_html('doc\pd-web-1.html')
t1=t1[0]
t1

,箱号,尺寸,箱型,进场日期,堆存天数,箱动态,箱状况,来箱情况,备注,堆场
0,箱号,尺寸,箱型,进场日期,堆存天数,箱动态,箱状况,来箱情况,备注,堆场
1,AMFU3238167,20,GP,2019-1-20 13:05:44,27,EC,坏箱,回空,NaN,SHIC
2,BMOU2241483,20,GP,2019-1-28 20:45:05,19,EC,坏箱,回空,NaN,SHIC
3,BMOU2405247,20,GP,2019-1-22 21:04:15,25,EC,好箱,洋山三期,NaN,SHWL
4,BMOU2552042,20,GP,2019-1-27 10:24:36,20,EC,坏箱,回空,NaN,SHIC
5,BMOU2553711,20,GP,2019-2-15 13:47:58,1,EC,坏箱,回空,NaN,SHIC


In [8]:
# pd-web-1.html的表格没有<thead>，使用header参数指定columns，index_col可以指定index
t2=pd.read_html('doc\pd-web-2.html',header=0,index_col=1)
t2=t2[0]
t2

,箱号,箱型,进场日期,堆存天数,箱动态,箱状况,来箱情况,备注,堆场
尺寸,,,,,,,,,
20,AMFU3238167,GP,2019-1-20 13:05:44,27,EC,坏箱,回空,NaN,SHIC
20,BMOU2241483,GP,2019-1-28 20:45:05,19,EC,坏箱,回空,NaN,SHIC
20,BMOU2405247,GP,2019-1-22 21:04:15,25,EC,好箱,洋山三期,NaN,SHWL
20,BMOU2552042,GP,2019-1-27 10:24:36,20,EC,坏箱,回空,NaN,SHIC
20,BMOU2553711,GP,2019-2-15 13:47:58,1,EC,坏箱,回空,NaN,SHIC


### XML
对XML进行解析，思路是通过支持解析XML的库，将XML文件解析为字典对象，然后再转换为DataFrame。  
比如使用`lxml.objectify`解析xml文件，先使用`getroot`获取文件的根节点root，root.[根节点名]返回产生各个XML元素的生成器。  
  
用例文件pd-xml.xml：
```XML
<class name = "一年级2班">
    <student>
        <name>王二麻子</name>
        <age>18</age>
        <sex>女</sex>
    </student>
    <student>
        <name>张三</name>
        <age>20</age>
        <sex>男</sex>
    </student>
    <student>
        <name>李四</name>
        <age>33</age>
        <sex>不知道</sex>
    </student>
</class>
```

In [9]:
from lxml import objectify
parsed=objectify.parse(open('doc\pd-xml.xml',encoding='utf-8'))
root=parsed.getroot()
print('根节点标签是：%s，标签属性值是：%s'%(root.tag,root.get('name'))) # root是根节点，即class节点，root.get()或root.attrib[]可以获取标签属性值

根节点标签是：class，标签属性值是：一年级2班


In [10]:
# 了解下如何对元素进行解析
student=root.student[0] # 第一个学生
info=student.getchildren() # 学生的信息列表，列表元素是每个子节点元素，包括tag和节点值
print('第一个节点标签是%s，内容是%s'%(info[0].tag,info[0].text))

第一个节点标签是name，内容是王二麻子


In [11]:
data=[]
for student in root.student: # 读取每个student节点
    info_data={}
    for info in student.getchildren(): # 读取student节点下每个子节点
        info_data[info.tag]=info.text # 节点标签和内容以字典形式保存
    data.append(info_data) # 将保存有学生信息的字典追加至data中
data # data为一个list，每个元素是保存有每个学生信息的字典

[{'name': '王二麻子', 'age': '18', 'sex': '女'},
 {'name': '张三', 'age': '20', 'sex': '男'},
 {'name': '李四', 'age': '33', 'sex': '不知道'}]

In [12]:
result=pd.DataFrame(data) # 将data转换为DataFrame
result

,age,name,sex
0,18,王二麻子,女
1,20,张三,男
2,33,李四,不知道


## API接口
许多网站都有公共API接口，提供json、xml或其他格式的数据。python可以通过访问这些API接口来获取数据，然后进行分析。访问API最简单的方法就是使用requests包。
我们通过requests发送一个HTTP GET请求到API接口地址，会接收到API接口返回的数据（一般是json或xml），根据数据格式使用相应的方法进行数据解析，获得python对象。
下面的例子访问github上pandas项目的issues，以获取issues的主题：

In [13]:
import requests
url='https://api.github.com/repos/pandas-dev/pandas/issues' # 接口url地址

In [14]:
resp=requests.get(url) # 发送GET请求到接口地址，并获取返回的json格式数据
resp # 200表明成功

<Response [200]>

In [15]:
data=resp.json() # 将json转换为字典格式

In [16]:
data[0]['title']

'Extension dtypes do not work with dataframe query method '

In [26]:
issues = pd.DataFrame(data, columns=['number', 'title', 'labels', 'state']) # 转换为DataFrame